
# bi-lstm+crf，使用pytorch-crf库实现crf，可cuda加速。

<font color='blac'>数据集说明：
实体说明：

<font color='blac'>&emsp;实体共有 52 种类型，均已经过脱敏处理；其中“O”为非实体。
标签中“B“代表一个实体的开始，”I“代表一个实体的中间或者结尾。”-“后的数字代号表示该
字符的实体类型。
### <font color='red'> !!!注意 label是1-54，但27， 45两个标签是没有的。<font color='blac'>

&emsp;值得注意的是实体不仅仅与实体词有关，而且与当前标题所售卖商品有关。举例说明，一
个售卖产品为手机壳的商品标题中出现的“iPhone13”与售卖产品为手机的商品标题中出现的
“iPhone13”为不同的实体标签。

#%

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from sklearn.model_selection import train_test_split
data = pd.read_excel('../datasets/sample_datasets/train_500.xlsx')
train_data, valid_data = train_test_split(data, test_size = 0.2, random_state=42)
train_data.index = list(range(len(train_data)))
valid_data.index = list(range(len(valid_data)))
# test_data = pd.read_csv('./test_public.csv')

## 看看文本长度的分布，以便确定padding的长度

In [4]:
len_text = data['text'].apply(lambda x: len(list(x)))
len_text.describe()


count    500.000000
mean      55.476000
std       13.212515
min        8.000000
25%       45.000000
50%       55.000000
75%       64.000000
max       94.000000
Name: text, dtype: float64

In [6]:
x = data['text'][0]
list(x)

['手',
 '机',
 '三',
 '脚',
 '架',
 '网',
 '红',
 '直',
 '播',
 '支',
 '架',
 '桌',
 '面',
 '自',
 '拍',
 '杆',
 '蓝',
 '牙',
 '遥',
 '控',
 '三',
 '脚',
 '架',
 '摄',
 '影',
 '拍',
 '摄',
 '拍',
 '照',
 '抖',
 '音',
 '看',
 '电',
 '视',
 '神',
 '器',
 '三',
 '角',
 '架',
 '便',
 '携',
 '伸',
 '缩',
 '懒',
 '人',
 '户',
 '外',
 '支',
 '撑',
 '架',
 ' ',
 '【',
 '女',
 '神',
 '粉',
 '】',
 '自',
 '带',
 '三',
 '脚',
 '架',
 '+',
 '蓝',
 '牙',
 '遥',
 '控']

['我', '爱', ' ', '你']